In [1]:
pip install alpaca-py

Note: you may need to restart the kernel to use updated packages.


In [2]:


from alpaca.data.historical import StockHistoricalDataClient, OptionHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, OptionChainRequest, StockLatestQuoteRequest, StockLatestTradeRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest, GetOptionContractsRequest
from alpaca.trading.enums import AssetClass, AssetExchange, AssetStatus

import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import math
import time

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.6f}'.format)

In [3]:
# Alpaca API key + secret
ALPACA_API_KEY = "PKKPF923IT8BIB4QYBBD"
ALPACA_SECRET = "4dVathgI73iQY7cmK6haZPnxjTDRFJbayIk4giqg"

## List of optionable securities

In [4]:
%%time

# Load Nasdaq's options list; includes other exchanges as well
options_url = "https://www.nasdaqtrader.com/dynamic/SymDir/options.txt"
options_df = pd.read_csv(options_url, sep="|", skipfooter=1, engine="python")
optionable_symbols = options_df["Underlying Symbol"].unique().tolist()

# Use the trading client to access asset metadata to filter
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET, paper=True)

# Get all active US equities
us_common_stocks = trading_client.get_all_assets(
    GetAssetsRequest(asset_class=AssetClass.US_EQUITY, status="active")
)

# Filter for exchange-listed stocks (not OTC, not crypto, not ETFs)
valid_equities = {
    asset.symbol for asset in us_common_stocks
    if asset.exchange in [AssetExchange.NASDAQ, AssetExchange.NYSE, AssetExchange.AMEX]
}

# Final symbol list = optionable and U.S. common stocks
filtered_symbols = sorted(set(optionable_symbols) & valid_equities)
len(filtered_symbols)

CPU times: user 3.95 s, sys: 555 ms, total: 4.51 s
Wall time: 24.3 s


3547

## Use historical data to compute volatility

In [5]:
# Download historical data
stock_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET)

In [6]:
end_date = (datetime.now() - timedelta(days=1)).date()
start_date = end_date - timedelta(days=90)

def fetch_close_prices(symbol_batch):
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol_batch,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date,
    )
    bars = stock_client.get_stock_bars(request_params)
    
    close_data = {}
    for symbol in symbol_batch:
        if symbol in bars:
            df = bars[symbol].df
            close_data[symbol] = df[['close']]
    return close_data

In [7]:
%%time
request_params = StockBarsRequest(
                        symbol_or_symbols=filtered_symbols,
                        timeframe=TimeFrame.Day,
                        #start=datetime(2025, 7, 11),
                        #end=datetime(2025, 7, 21),
                        start=start_date,
                        end=end_date
                 )

bars = stock_client.get_stock_bars(request_params)
df_historical = bars.df
df_historical.shape

CPU times: user 3.31 s, sys: 120 ms, total: 3.43 s
Wall time: 16.3 s


(216019, 7)

In [8]:
# Drop symbols with not enough records
record_count_by_ticker = df_historical.groupby(level='symbol').size().sort_values()
symbols_to_keep = record_count_by_ticker[record_count_by_ticker>57].index.tolist()
df_historical_clean = df_historical.loc[df_historical.index.get_level_values('symbol').isin(symbols_to_keep)]

# Turn index into columns
df_historical_clean = df_historical_clean.reset_index()
df_historical_clean.shape

(215572, 9)

In [9]:
def compute_hv_per_symbol(df, trading_days=252):
    """
    Computes historical volatility (annualized) for each symbol in the DataFrame.

    Parameters:
        df (pd.DataFrame): Must contain 'symbol', 'timestamp', and 'close' columns.
        trading_days (int): Number of trading days in a year (default: 252)

    Returns:
        pd.DataFrame: DataFrame with 'symbol' and 'volatility' columns.
    """
    def compute_hv(group):
        prices = group['close'].dropna()
        log_returns = np.log(prices / prices.shift(1)).dropna()
        if len(log_returns) < 2:
            return np.nan
        return log_returns.std(ddof=0) * np.sqrt(trading_days)

    # Group by symbol and compute volatility
    hv_df = df.groupby('symbol').apply(compute_hv).reset_index(name='volatility')
    return hv_df


df_historical_volatility = compute_hv_per_symbol(df_historical_clean)
df_historical_volatility.shape

(3534, 2)

## Get latest trade price for every stock

In [10]:


resp = stock_client.get_stock_latest_trade(
    StockLatestTradeRequest(symbol_or_symbols=symbols_to_keep)
)


# Convert Alpaca model objects to plain dicts
flattened = {symbol: trade.dict() for symbol, trade in resp.items()}

# Create DataFrame
df_latest_trades = pd.DataFrame.from_dict(flattened, orient='index')

# Add the symbol as a column
df_latest_trades.reset_index(inplace=True, drop=True)

# rename columns
df_latest_trades.rename(columns={'price': 'underlying_price'}, inplace=True)


df_latest_trades.shape

/var/folders/j5/8wd0ptln5b3561c3f8zcw19w0000gr/T/ipykernel_7845/3848459258.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  flattened = {symbol: trade.dict() for symbol, trade in resp.items()}


(3534, 8)

In [11]:
df_latest_trades.timestamp.max()

Timestamp('2025-07-23 13:45:20.863464+0000', tz='UTC')

## Options data

In [12]:
def get_all_option_contracts(trading_client, 
                              underlying_symbols,
                              status=AssetStatus.ACTIVE,
                              expiration_date = '2025-08-15',
                              type = 'put',
                              limit=10000,
                              sleep_sec=0.25):
    """
    Fetches all option contracts for a list of underlying symbols using pagination.
    
    Parameters:
        trading_client: An authenticated Alpaca TradingClient
        underlying_symbols: List of tickers (e.g., ["AAPL", "MSFT"])
        status: AssetStatus (default: ACTIVE)
        limit: Max results per page (default: 100)
        sleep_sec: Pause between paginated requests to avoid hitting rate limits
        filters: Additional filters like expiration_date_gte, type, etc.
    
    Returns:
        A list of OptionContract objects
    """
    all_contracts = []
    page_token = None
    
    counter = 0

    while True:
        counter+=1
        req = GetOptionContractsRequest(
            underlying_symbols=underlying_symbols,
            status=status,
            expiration_date = expiration_date,
            type = type,
            limit=limit,
            page_token=page_token
        )
        res = trading_client.get_option_contracts(req)

        all_contracts.extend(res.option_contracts)

        if res.next_page_token:
            page_token = res.next_page_token
            time.sleep(sleep_sec)
        else:
            break

    return all_contracts


def flatten_option_list(option_list):
    flat_records = []

    for option in option_list:
        flat_records.append({
            'symbol': option.symbol,
            'name': option.name,
            'underlying_symbol': option.underlying_symbol,
            'strike_price': option.strike_price,
            'expiration_date': option.expiration_date,
            'close_price': float(option.close_price) if option.close_price else None,
            'close_price_date': option.close_price_date,
            'open_interest': int(option.open_interest) if option.open_interest else None,
            'open_interest_date': option.open_interest_date,
            'style': str(option.style.value) if option.style else None,
            'type': str(option.type.value) if option.type else None,
            'status': str(option.status.value) if option.status else None,
            'tradable': option.tradable,
            'size': int(option.size) if option.size else None,
            'id': str(option.id),
        })

    return pd.DataFrame(flat_records)

In [13]:
# Get open interest from contract data
contracts = get_all_option_contracts(trading_client, symbols_to_keep)

# keep only contracts with open_interest above some threshold
open_interest_minimum = 0
open_interest_contracts = []
for contract in contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest_minimum):
        open_interest_contracts.append(contract)
        
options_symbol_list = [dict(option)['symbol'] for option in open_interest_contracts]

        
df_options_contracts = flatten_option_list(open_interest_contracts)
df_options_contracts = df_options_contracts[df_options_contracts['tradable']==True]
df_options_contracts.shape

(31949, 15)

In [14]:
# Get greeks, bids, asks on options chains

# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET, url_override = None)

In [15]:
# get option chain by underlying_symbol
#req = OptionChainRequest(underlying_symbol = 'AAPL', 
#                        type = 'put',
#                        expiration_date = '2025-08-15'
#                        )
#options_chain_res = option_historical_data_client.get_option_chain(req)


In [16]:
%%time
counter = 0 
results = {}

for symbol in symbols_to_keep:
    print(counter, symbol)
    try:
        req = OptionChainRequest(underlying_symbol = symbol, 
                        type = 'put',
                        expiration_date = '2025-08-15'
                        )
        options_chain_res = option_historical_data_client.get_option_chain(req)
        print(len(options_chain_res.keys()), 'records', '\n')

        results.update(options_chain_res)  # merges each symbol's data into the main dict
    except Exception as e:
        print(f"Failed to get data for {symbol}: {e}")
    
    counter+=1
    #time.sleep(0.25)

0 APPX
27 records 

1 OTEX
14 records 

2 OSS
4 records 

3 OSPN
11 records 

4 OTIS
27 records 

5 OTLK
7 records 

6 OSW
12 records 

7 OSUR
3 records 

8 A
31 records 

9 OTTR
15 records 

10 OUST
47 records 

11 OSK
23 records 

12 OVID
3 records 

13 OVV
21 records 

14 OWL
35 records 

15 OXM
9 records 

16 OXSQ
3 records 

17 OXY
48 records 

18 OZEM
21 records 

19 OZK
20 records 

20 OTLY
17 records 

21 OUT
19 records 

22 OS
12 records 

23 OSCR
44 records 

24 OPI
7 records 

25 OPK
5 records 

26 OPRA
12 records 

27 OPRT
6 records 

28 OPRX
10 records 

29 OPTT
7 records 

30 OPY
13 records 

31 OR
7 records 

32 ORA
17 records 

33 ORC
5 records 

34 ORCL
65 records 

35 ORGN
7 records 

36 ORGO
3 records 

37 ORI
9 records 

38 ORIC
17 records 

39 ORKA
10 records 

40 ORLA
9 records 

41 ORLY
146 records 

42 ORMP
3 records 

43 ORN
7 records 

44 ORRF
9 records 

45 PAA
35 records 

46 OSBC
11 records 

47 OSIS
28 records 

48 PAAS
35 records 

49 PAGP
21 records 

50

13 records 

393 PRDO
8 records 

394 PROF
5 records 

395 PROP
3 records 

396 PRPH
7 records 

397 PRPL
3 records 

398 PRQR
3 records 

399 PRSU
8 records 

400 PRTA
4 records 

401 PRTH
6 records 

402 PRTS
3 records 

403 PRU
20 records 

404 PRVA
11 records 

405 PSA
29 records 

406 PROK
12 records 

407 PEPG
9 records 

408 PRCT
21 records 

409 PRAX
27 records 

410 PMVP
3 records 

411 PNC
52 records 

412 PNFP
28 records 

413 PNNT
5 records 

414 PNR
32 records 

415 PNW
18 records 

416 PODD
29 records 

417 POET
23 records 

418 PONY
36 records 

419 POOL
29 records 

420 POR
10 records 

421 PRCH
8 records 

422 POST
22 records 

423 POWL
50 records 

424 POWW
4 records 

425 PPC
18 records 

426 PPG
52 records 

427 PPIH
8 records 

428 PPL
21 records 

429 PPSI
9 records 

430 PPTA
11 records 

431 PR
22 records 

432 PRA
11 records 

433 PRAA
12 records 

434 POWI
12 records 

435 NSIT
31 records 

436 NRP
19 records 

437 NSA
8 records 

438 LWLG
8 records 

439 LX
8

20 records 

781 MNRO
11 records 

782 MNSO
12 records 

783 MNST
37 records 

784 MNTK
3 records 

785 MNTN
9 records 

786 MNY
3 records 

787 MO
38 records 

788 MODG
7 records 

789 MOFG
8 records 

790 MOGO
7 records 

791 MOH
38 records 

792 MOMO
16 records 

793 MORN
29 records 

794 MMC
25 records 

795 MLYS
10 records 

796 MLTX
16 records 

797 MLSS
3 records 

798 MGM
41 records 

799 MGNI
27 records 

800 MGNX
7 records 

801 MGPI
9 records 

802 MGTX
6 records 

803 MGY
8 records 

804 MHK
28 records 

805 MHO
23 records 

806 MIDD
23 records 

807 MIR
13 records 

808 MIRM
11 records 

809 MOS
37 records 

810 MITK
8 records 

811 MKC
15 records 

812 MKL
49 records 

813 MKSI
19 records 

814 MKTW
6 records 

815 MKTX
34 records 

816 MLCO
25 records 

817 MLI
15 records 

818 MLKN
11 records 

819 MLM
29 records 

820 MLNK
11 records 

821 MLR
10 records 

822 MITT
14 records 

823 MOV
11 records 

824 MP
65 records 

825 MPAA
8 records 

826 MSGS
33 records 

827 MSI


5 records 

1160 ZETA
36 records 

1161 ZEUS
8 records 

1162 ZG
19 records 

1163 ZGN
7 records 

1164 ZH
3 records 

1165 ZIM
45 records 

1166 ZIMV
12 records 

1167 ZION
15 records 

1168 ZIP
4 records 

1169 ZK
7 records 

1170 ZKIN
6 records 

1171 ZM
51 records 

1172 ZNTL
3 records 

1173 ZS
70 records 

1174 ZTO
19 records 

1175 ZTS
23 records 

1176 ZUMZ
11 records 

1177 ZURA
3 records 

1178 ZVIA
3 records 

1179 ZVRA
23 records 

1180 ZWS
16 records 

1181 ZD
9 records 

1182 ZBRA
55 records 

1183 ZBH
18 records 

1184 Z
52 records 

1185 XOM
51 records 

1186 XOMA
8 records 

1187 XP
37 records 

1188 XPEL
14 records 

1189 XPER
6 records 

1190 XPEV
37 records 

1191 XPO
37 records 

1192 XPOF
8 records 

1193 XPRO
7 records 

1194 XRAY
11 records 

1195 XRX
11 records 

1196 XMTR
23 records 

1197 XYL
23 records 

1198 YALA
5 records 

1199 YELP
24 records 

1200 YETI
17 records 

1201 YEXT
18 records 

1202 YMAB
7 records 

1203 YMM
9 records 

1204 YOU
47 records 



10 records 

1530 RTO
8 records 

1531 RTX
41 records 

1532 RUM
28 records 

1533 RUN
37 records 

1534 RUNN
21 records 

1535 RUSHA
12 records 

1536 RVLV
7 records 

1537 RVMD
23 records 

1538 RVPH
7 records 

1539 RVTY
19 records 

1540 RWAY
8 records 

1541 RSKD
4 records 

1542 SLGN
11 records 

1543 SLI
7 records 

1544 SLM
23 records 

1545 SW
11 records 

1546 SWBI
17 records 

1547 SWIM
6 records 

1548 SWK
22 records 

1549 SWKS
38 records 

1550 SXC
6 records 

1551 SXI
33 records 

1552 SY
4 records 

1553 SYBT
15 records 

1554 SYF
29 records 

1555 SYK
33 records 

1556 SYM
48 records 

1557 SYNA
13 records 

1558 SYRE
11 records 

1559 SYY
24 records 

1560 T
37 records 

1561 TAC
18 records 

1562 TACT
3 records 

1563 TAK
11 records 

1564 TAL
31 records 

1565 TALK
8 records 

1566 TALO
7 records 

1567 TAP
13 records 

1568 SVV
8 records 

1569 SVRA
6 records 

1570 SVM
3 records 

1571 SVC
7 records 

1572 STLD
29 records 

1573 STM
33 records 

1574 STNE
33 recor

28 records 

1900 BMI
33 records 

1901 BMEA
5 records 

1902 BMBL
13 records 

1903 BMA
15 records 

1904 BK
40 records 

1905 BKD
13 records 

1906 BKE
16 records 

1907 BKH
13 records 

1908 BKNG
123 records 

1909 BKR
25 records 

1910 BKSY
13 records 

1911 BKU
12 records 

1912 BKV
11 records 

1913 BL
26 records 

1914 BLBD
9 records 

1915 BRBS
3 records 

1916 BLCO
21 records 

1917 BLDP
7 records 

1918 BLDR
34 records 

1919 BLFY
7 records 

1920 BLK
89 records 

1921 BLKB
12 records 

1922 BLMN
7 records 

1923 BLND
10 records 

1924 BLNK
4 records 

1925 BLRX
22 records 

1926 BLX
9 records 

1927 BLZE
4 records 

1928 BLDE
3 records 

1929 BRC
17 records 

1930 BRCC
5 records 

1931 BRDG
7 records 

1932 BUD
39 records 

1933 BUG
21 records 

1934 BULL
43 records 

1935 BUR
11 records 

1936 BURL
51 records 

1937 BV
11 records 

1938 BVN
18 records 

1939 BVS
5 records 

1940 BWA
14 records 

1941 BWB
11 records 

1942 BWEN
3 records 

1943 BTU
31 records 

1944 BWIN
10 

4 records 

2268 AON
31 records 

2269 AORT
9 records 

2270 AOS
13 records 

2271 AOSL
12 records 

2272 AOUT
9 records 

2273 AP
3 records 

2274 APA
33 records 

2275 APAM
9 records 

2276 APD
29 records 

2277 APEI
9 records 

2278 APGE
11 records 

2279 APH
19 records 

2280 API
3 records 

2281 APLD
42 records 

2282 APLS
19 records 

2283 APLT
3 records 

2284 APO
41 records 

2285 APOG
20 records 

2286 APP
105 records 

2287 ANIX
3 records 

2288 ANIP
13 records 

2289 ANIK
9 records 

2290 ANGO
8 records 

2291 AMP
29 records 

2292 AMPL
10 records 

2293 AMPX
15 records 

2294 AMPY
7 records 

2295 AMR
42 records 

2296 AMRC
12 records 

2297 AMRK
7 records 

2298 AMRN
34 records 

2299 AMRX
6 records 

2300 AMSC
44 records 

2301 AMT
32 records 

2302 AMLX
11 records 

2303 AMTB
11 records 

2304 AMWD
11 records 

2305 AMWL
7 records 

2306 AMX
20 records 

2307 AMZD
23 records 

2308 AMZN
64 records 

2309 AMZU
30 records 

2310 AN
22 records 

2311 ANAB
12 records 

2312 

3 records 

2638 ATHM
10 records 

2639 ATI
20 records 

2640 ATKR
16 records 

2641 ATLO
12 records 

2642 ATLX
4 records 

2643 ATMU
13 records 

2644 ATNI
11 records 

2645 ATNM
3 records 

2646 ATO
26 records 

2647 ATEX
11 records 

2648 CXW
19 records 

2649 CYBN
8 records 

2650 CYBR
50 records 

2651 HGTY
8 records 

2652 HGV
11 records 

2653 HHH
15 records 

2654 HI
8 records 

2655 HIG
23 records 

2656 HII
26 records 

2657 HIMS
66 records 

2658 HIMX
17 records 

2659 HIMZ
55 records 

2660 HIPO
10 records 

2661 HITI
3 records 

2662 HIVE
10 records 

2663 HIW
9 records 

2664 HL
23 records 

2665 HLF
29 records 

2666 HLIO
9 records 

2667 HLIT
7 records 

2668 HLLY
3 records 

2669 HLMN
5 records 

2670 HLN
7 records 

2671 HLNE
31 records 

2672 HLT
52 records 

2673 HLX
14 records 

2674 HG
7 records 

2675 HFWA
8 records 

2676 HFFG
3 records 

2677 HESM
24 records 

2678 HAL
37 records 

2679 HALO
11 records 

2680 HAS
29 records 

2681 HASI
8 records 

2682 HAYW
19

12 records 

3009 HYMC
7 records 

3010 HZO
8 records 

3011 IAC
14 records 

3012 IAG
15 records 

3013 IART
10 records 

3014 IAS
6 records 

3015 IBB
39 records 

3016 IBCP
10 records 

3017 IBIT
86 records 

3018 IBKR
10 records 

3019 IBM
60 records 

3020 IMPP
7 records 

3021 IBN
17 records 

3022 IBRX
16 records 

3023 IBTA
10 records 

3024 ICE
22 records 

3025 ICHR
19 records 

3026 ICL
5 records 

3027 ICLN
32 records 

3028 ICLR
27 records 

3029 ICUI
32 records 

3030 IDA
19 records 

3031 IDCC
26 records 

3032 IDN
10 records 

3033 IBOC
14 records 

3034 IMTX
12 records 

3035 IMUX
7 records 

3036 IMVT
21 records 

3037 INTU
50 records 

3038 INTW
16 records 

3039 INVA
9 records 

3040 INVE
3 records 

3041 INVH
14 records 

3042 INVX
11 records 

3043 INVZ
6 records 

3044 IONL
35 records 

3045 IONQ
62 records 

3046 IONS
15 records 

3047 IOSP
13 records 

3048 INTT
5 records 

3049 IOT
33 records 

3050 IP
41 records 

3051 IPA
7 records 

3052 IPG
19 records 

30

7 records 

3379 GCT
12 records 

3380 GANX
6 records 

3381 GD
58 records 

3382 GDEN
8 records 

3383 GDOT
8 records 

3384 GDRX
3 records 

3385 GDS
34 records 

3386 GDYN
9 records 

3387 GE
63 records 

3388 GEF
14 records 

3389 GEHC
42 records 

3390 GEL
11 records 

3391 GEN
20 records 

3392 GENC
11 records 

3393 GDDY
48 records 

3394 FLGT
10 records 

3395 GAMB
11 records 

3396 GABC
10 records 

3397 FTCI
4 records 

3398 FTDR
12 records 

3399 FTEK
6 records 

3400 FTHM
3 records 

3401 FTI
33 records 

3402 FTK
19 records 

3403 FTNT
52 records 

3404 FTRE
15 records 

3405 FTS
11 records 

3406 FTV
30 records 

3407 FUBO
19 records 

3408 GALT
11 records 

3409 FUL
16 records 

3410 FULT
11 records 

3411 FUN
9 records 

3412 FUTU
61 records 

3413 FVCB
9 records 

3414 FVRR
30 records 

3415 FWONA
18 records 

3416 FWONK
22 records 

3417 FWRD
8 records 

3418 FWRG
13 records 

3419 FYBR
14 records 

3420 G
10 records 

3421 FULC
15 records 

3422 KOPN
7 records 

3423

In [17]:
def flatten_option_chain(option_data):
    records = []

    for symbol, snapshot in option_data.items():
        quote = snapshot.latest_quote
        trade = snapshot.latest_trade
        greeks = snapshot.greeks

        record = {
            'option_symbol': symbol,
            'bid_price': quote.bid_price if quote else None,
            'bid_size': quote.bid_size if quote else None,
            'ask_price': quote.ask_price if quote else None,
            'ask_size': quote.ask_size if quote else None,
            'quote_timestamp': quote.timestamp if quote else None,

            'trade_price': trade.price if trade else None,
            'trade_size': trade.size if trade else None,
            'trade_timestamp': trade.timestamp if trade else None,

            'implied_volatility': snapshot.implied_volatility,

            # Break out Greeks
            'delta': greeks.delta if greeks else None,
            'gamma': greeks.gamma if greeks else None,
            'theta': greeks.theta if greeks else None,
            'vega': greeks.vega if greeks else None,
            'rho': greeks.rho if greeks else None,
        }

        records.append(record)

    return pd.DataFrame(records)

In [18]:
df_options = flatten_option_chain(results)
df_options['ticker'] = df_options['option_symbol'].str.extract(r'^([A-Z]+)')

# keep only options found above
df_options_filtered = df_options[df_options['option_symbol'].isin(options_symbol_list)].reset_index(drop=True)
df_options_filtered.shape

(32162, 16)

## Assemble the pieces

In [19]:
df_merged_all = pd.merge(
    pd.merge(
        pd.merge(
            df_options_contracts,
            df_options_filtered,
            left_on='symbol',
            right_on='option_symbol',
            how='inner'
        ),
        df_latest_trades,
        left_on='underlying_symbol',
        right_on='symbol',
        how='inner',
        suffixes=('', '_latest')
    ),
    df_historical_volatility,
    left_on='underlying_symbol',
    right_on='symbol',
    how='inner',
    suffixes=('', '_latest_2')
)


# Append time till expiry
df_merged_all['expiration_date'] = pd.to_datetime(df_merged_all['expiration_date'])

# Compute days till expiry
today = datetime.today()
df_merged_all['days_till_expiry'] = (df_merged_all['expiration_date'] - today).dt.days

# Convert to years
df_merged_all['time_till_expiry'] = df_merged_all['days_till_expiry'] / 365


#append risk free rate
risk_free_rate = .0435
df_merged_all['risk_free_rate'] = risk_free_rate

df_merged_all.shape

(31949, 44)

## Probability of success

In [20]:
def probability_put_expires_worthless(S, K, T, r, vol):
    d2 = (math.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * math.sqrt(T))
    return norm.cdf(d2)

In [21]:
df_merged_all['prob_put_expires_worthless'] = df_merged_all.apply(
    lambda row: probability_put_expires_worthless(
        row['underlying_price'],
        row['strike_price'],
        row['time_till_expiry'],
        row['risk_free_rate'],
        row['volatility']
    ),
    axis=1
)

df_merged_all.shape

(31949, 45)

## Clean up final output

In [22]:
# clean up unnecessary columns collected from the various API responses
cols_to_drop = ['open_interest_date', 'style', 'tradable', 'status', 'size', 'id', 'symbol', 'bid_size', 'ask_size', 'quote_timestamp', 
                'trade_size', 'trade_timestamp', 'ticker', 'symbol_latest', 'timestamp', 'exchange', 'size_latest', 
                'id_latest', 'conditions', 'tape', 'symbol_latest_2', 'time_till_expiry', 'risk_free_rate', 
                'ask_price', 'trade_price', 'close_price', 'close_price_date',
                'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho' # get rid of these for now
               ]

df_merged_all.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# rename columns
df_merged_all.rename(columns={'bid_price': 'option_bid_price'}, inplace=True)


df_merged_all.shape

(31949, 12)

In [23]:
# after hours, bids are mostly zero. 
# If this is run during trading hours, use bid_price
# If this is run after hours, use trade_price. This might be inaccurate on sparsely traded options
df_merged_all['contract_revenue'] = df_merged_all['option_bid_price'] * 100 
#df_merged_all['contract_revenue'] = df_merged_all['trade_price'] * 100 

df_merged_all['collateral'] = df_merged_all['strike_price'] * 100
df_merged_all['potential_return'] = df_merged_all['contract_revenue'] / df_merged_all['collateral']

In [24]:
# Round columns: 2 decimals or 4 decimals
round_cols_2 = ['strike_price','close_price','bid_price','ask_price','trade_price','underlying_price','contract_revenue','collateral']
round_cols_4 = ['implied_volatility','delta','gamma','theta','vega','rho','volatility','prob_put_expires_worthless','potential_return']

# Only round columns that exist
cols_2 = [col for col in round_cols_2 if col in df_merged_all.columns]
cols_4 = [col for col in round_cols_4 if col in df_merged_all.columns]

df_merged_all[cols_2] = df_merged_all[cols_2].round(2)
df_merged_all[cols_4] = df_merged_all[cols_4].round(4)


In [25]:
# filter
df_merged_all = df_merged_all[df_merged_all['prob_put_expires_worthless'] > .70]
df_merged_all = df_merged_all[df_merged_all['collateral'] <= 15000]
df_merged_all = df_merged_all[df_merged_all['option_bid_price'] > .10]
df_merged_all = df_merged_all[df_merged_all['open_interest']>=100]

df_merged_all = df_merged_all.sort_values('potential_return', ascending=False).drop_duplicates(subset='underlying_symbol', keep='first')
df_merged_all.shape

(960, 15)

In [26]:
df_merged_all.to_excel('output20250815.xlsx', index=False)